In [1]:
##Installing all the necessary modules to run the following code. 
#Make sure to install thunderpack if not yet installed: "pip install thunderpack".
#Steps 1-5 are optional, only if you want to recreate a new sampling cohort. This will randomly select patient notes. 
#Steps 1-25 are if you want to recreate the data from this study using the sampling cohorts we made using optioanl steps 1-5 with randomization.  


In [ ]:
#Optional step 1 creates the samping cohort for positive ICD codes for Beth Israel Deaconess Medical Center (BIDMC)
#Reminder: ICD+ group is defined as patients who received at least one SDH-related ICD.

In [ ]:
#Import the nessary modules
import numpy as np
import pandas as pd
from tqdm import tqdm
from thunderpack import ThunderReader  
import gc

In [2]:
#Read the thunderpack file.
reader = ThunderReader('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/thunderpack_icd_9_10_nax_1m_BIDMC')

dfs = []

#Creating a loop that goes over all the keys to search for whether or not it contains the SDH+ ICD codes.
#Concatenate all filtered dataframes from all partitions into one dataframe.
for i in tqdm(range(1, (len(list(reader.keys()))+1))):  
    df = reader[f'ICD_partition_{i}']
    df = df[df.DiagnosisCodeWithDots.astype(str).str.contains('^(?:I62.0|S06.5|432.1|852.2|852.3)')]  
    dfs.append(df)
df_icd_plus_BI = pd.concat(dfs, axis=0, ignore_index=True)  

#Keep the unique patient IDs and save them as a CSV file.
df_icd_plus_BI = df_icd_plus_BI[['BDSPPatientID','AdmissionDate','BDSPEncounterID', 'DiagnosisCodeWithDots']].drop_duplicates()
df_icd_plus_BI.to_csv('patientIDs_ICD_plus_SDH_BI.csv', index=False)    


100%|██████████| 36/36 [00:47<00:00,  1.32s/it]


In [3]:
#Load in the positive ICD group for BIDMC.
df_icd_plus_BI = pd.read_csv('patientIDs_ICD_plus_SDH_BI.csv') 

#Reminder: The ICD minus group is defined as patients who never have received any SDH related ICDs.
dfs = []

#Creating a  loop that goes over all the keys to search for whether it contains the following ICD+ codes so it can exclude those with ICD+.
for i in tqdm(range(1, (len(list(reader.keys()))+1))):   
    df = reader[f'ICD_partition_{i}']
    df = df[~df.DiagnosisCodeWithDots.astype(str).str.contains('^(?:I62.0|S06.5|432.1|852.2|852.3)')]  
    df = df[['BDSPPatientID','AdmissionDate','BDSPEncounterID','DiagnosisCodeWithDots']]
    dfs.append(df)

    #To further save space, delete the df.
    del df
    gc.collect()

#Concatenate all filtered dataframes from all partitions into one dataframe.
df_icd_minus_BI = pd.concat(dfs, axis=0, ignore_index=True)  
df_icd_minus_BI = df_icd_minus_BI[['BDSPPatientID','AdmissionDate','BDSPEncounterID']].drop_duplicates()

#Conceptually the following code can be described as df_icd_minus = df_icd_minus - df_icd_plus
df_icd_minus_BI = df_icd_minus_BI[ ~np.in1d(df_icd_minus_BI.BDSPPatientID, df_icd_plus_BI.BDSPPatientID) ]

#Save as a CSV file
df_icd_minus_BI.to_csv('patientIDs_ICD_minus2_SDH_BI.csv', index=False)   



  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:32<00:00,  1.11it/s]
